In [17]:
deployment = "gpt4"
model = "gpt-4"

# 大模型辅助设计

In [7]:
import openai
def work_on(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a senior software engineer."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 500
      )
    return response.choices[0].message.content

In [14]:
prompt = """
Design a microservice according to the following requirements:
The service will process the user's input and store the result. 
After processing a token will be returned to the user, it would be used to get the processing result.
Consider on the following design principles:
1. concerns separation, try you best to separate the functions into the small modules.
2. single responsibility 
3. interface oriented
make sure the extensibility and reusability of the design, represent your design (with main abstractions) with PlantUML script
"""

In [15]:
print(work_on(prompt))

Here is the PlantUML description for the design of the microservice:

```uml
@startuml
title User Input Processing Microservice

package "User Input Processing Microservice" {

interface "IUserController" {
    + postProcess(input: UserInput) : Token
    + getProcessingResult(token: Token) : ProcessingResult
}

class UserController {
    - userService : IUserService
    + postProcess(input: UserInput) : Token
    + getProcessingResult(token: Token) : ProcessingResult
}

interface "IUserService" {
    + processInput(input: UserInput) : Token
    + getResult(token: Token) : ProcessingResult
}

class UserService {
    - processingRepository: IProcessingRepository
    + processInput(input: UserInput) : Token
    + getResult(token: Token) : ProcessingResult
}

interface "IProcessingRepository" {
    + saveProcessing(input: UserInput) : Token
    + fetchProcessing(token: Token) : ProcessingResult
}

class ProcessingRepository {
    + saveProcessing(input: UserInput) : Token
    + fetchProces

# 生成Plugin代码及Skeleton单元测试

In [18]:
import os
import json
import re

def find_declarations(directory, json_string):
    # Parse the JSON string
    json_data = json.loads(json_string)
    package_name = json_data["package_name"]
    element_name = json_data["element"]

    # Regular expression to match structure/interface declarations
    regex = re.compile(r"(type\s+" + element_name + r"\s+(struct|interface)\s+{\s+[^}]*\s+})")

    # Read the source files in the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".go"):
                with open(os.path.join(root, file), "r") as f:
                    content = f.read()
                    # Check if the file is in the correct package
                    if "package " + package_name in content:
                        # Search for the structure/interface declaration
                        match = regex.search(content)
                        if match:
                            return match.group(1)

    return "Not Found"

In [19]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [20]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMRequestsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
import time
import os
import json
import re

src_dir = "./llm_friendly_code"

def find_declarations(directory, json_string):
    # Parse the JSON string
    json_data = json.loads(json_string)
    package_name = json_data["package_name"]
    element_name = json_data["element"]

    # Regular expression to match structure/interface declarations
    regex = re.compile(r"(type\s+" + element_name + r"\s+(struct|interface)\s+{\s+[^}]*\s+})")

    # Read the source files in the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".go"):
                with open(os.path.join(root, file), "r") as f:
                    content = f.read()
                    # Check if the file is in the correct package
                    if "package " + package_name in content:
                        # Search for the structure/interface declaration
                        match = regex.search(content)
                        if match:
                            return match.group(1)

    return "Not Found"

def get_dep(json_string):
    return find_declarations(src_dir, json_string)

llm = AzureChatOpenAI(deployment_name=deployment, temperature=0)

# 将“依赖获取”工具添加到 ReAct 过程中    
tools = [
    Tool(
        name="find_declarations", func=get_dep,
        description="""Only invoke it when you need to find the declaration of the interface/struct/method that is not in the given code. 
        The input is a JSON string includes package name and the struct/interface name, 
          and its the format {{"package_name":<package name>,"element":<struct/interface>}}
        """)
]

In [21]:
def write_code(input):
    agent = initialize_agent(tools, llm, agent="zero-shot-react-description", handle_parsing_errors=True, max_iterations=10, verbose=True)
    print(agent.run(input))

In [22]:
write_code(
"""
 You are a very senior golang programmer.
 Write the golang code to implement the demo.TokenCreator interface.
 The implementation is to generate the token with uuid and its package is demo
"""
)



> Entering new AgentExecutor chain...
First, I need to know what methods are in the demo.TokenCreator interface.
Action: find_declarations
Action Input: {"package_name":"demo","element":"TokenCreator"}
Observation: type TokenCreator interface {
	CreateToken(data string) string
}
Thought:I now know the final answer
Final Answer: 
Here is the implementation of the demo.TokenCreator interface in Go:

```go
package demo

import (
	"github.com/google/uuid"
)

type TokenCreatorImpl struct{}

func (t *TokenCreatorImpl) CreateToken(data string) string {
	return uuid.New().String() + data
}
```

In this implementation, we are using the google/uuid package to generate a new UUID. The CreateToken method concatenates this UUID with the input data to create a token.

> Finished chain.
Here is the implementation of the demo.TokenCreator interface in Go:

```go
package demo

import (
	"github.com/google/uuid"
)

type TokenCreatorImpl struct{}

func (t *TokenCreatorImpl) CreateToken(data string) str

In [23]:
write_code(
"""
 You are a very senior golang programmer.
 Write the golang code to implement the demo.Storage interface.
 The implementation is based on local memory.
"""
)



> Entering new AgentExecutor chain...
First, I need to know what the demo.Storage interface looks like in order to implement it. 
Action: find_declarations
Action Input: {"package_name":"demo","element":"Storage"} 
Observation: type Storage interface {
	// RetiveData is to retrive the data by the associated token.
	RetiveData(token string) (string, error)

	// StoreData is to persist the data,
	// input paramters:
	//   token is used to retrive the associated data
	//
	StoreData(token string, data string) error
}
Thought:Now that I know the methods that the Storage interface has, I can create a struct that implements this interface. The struct will need a field to store the data in memory. I'll use a map for this, with the token as the key and the data as the value. 
Final Answer: 
```go
package demo

import "errors"

type MemoryStorage struct {
	data map[string]string
}

func NewMemoryStorage() *MemoryStorage {
	return &MemoryStorage{
		data: make(map[string]string),
	}
}

func (m *

In [40]:
write_code(
"""
 You are a very senior golang programmer.
 Write the golang code to implement the demo.Processor interface.
 The implementation is to reverse the input content
"""
)



> Entering new AgentExecutor chain...
I need to know the definition of the demo.Processor interface to implement it.
Action: find_declarations
Action Input: {"package_name":"demo","element":"Processor"} 
Observation: type Processor interface {
	// Process is to the raw data
	Process(raw string) (string, error)
}
Thought:I now know the final answer
Final Answer: 
Here is the Go code to implement the demo.Processor interface. The implementation reverses the input content.

```go
package main

import (
	"errors"
	"strings"
	"demo"
)

type ReverseProcessor struct{}

func (rp *ReverseProcessor) Process(raw string) (string, error) {
	if raw == "" {
		return "", errors.New("input string is empty")
	}

	runes := []rune(raw)
	for i, j := 0, len(runes)-1; i < j; i, j = i+1, j-1 {
		runes[i], runes[j] = runes[j], runes[i]
	}

	return string(runes), nil
}

func main() {
	var p demo.Processor = &ReverseProcessor{}
	result, err := p.Process("Hello, World!")
	if err != nil {
		panic(err)
	}
	fmt.Pr

In [24]:
def write_unit_tests(filepath):
    agent = initialize_agent(tools, llm, agent="zero-shot-react-description", handle_parsing_errors=True, max_iterations=10, verbose=True)
    return (agent.run("""
    You are a very senior golang programmer.
    Please, follow the instructions:
     1. consider on the edge cases and errors
     2. make sure the generated unit test code in the same package as the tested code.
    write an unit tests for the following code: \n ---""" + read_file(filepath)
     ))

In [25]:
print(write_unit_tests(src_dir+"/demo/skeleton.go"))



> Entering new AgentExecutor chain...
I need to write unit tests for the methods Process and Retrieve of the ProcessingServiceImpl struct. To do this, I need to understand the interfaces Storage, Processor, and TokenCreator that are used in these methods. I will use the find_declarations tool to find the declarations of these interfaces.
Action: find_declarations
Action Input: {"package_name":"demo","element":"Storage"}
Observation: type Storage interface {
	// RetiveData is to retrive the data by the associated token.
	RetiveData(token string) (string, error)

	// StoreData is to persist the data,
	// input paramters:
	//   token is used to retrive the associated data
	//
	StoreData(token string, data string) error
}
Thought:I now know the Storage interface has two methods, RetiveData and StoreData. I will now find the declarations for the Processor and TokenCreator interfaces.
Action: find_declarations
Action Input: {"package_name":"demo","element":"Processor"}
Observation: type Pr